### 기본 세팅

In [56]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
import time
import pandas as pd
import re

In [57]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [125]:
driver = webdriver.Chrome(options=chrome_options)

In [109]:
driver = webdriver.Chrome()

In [119]:
driver.quit()

In [59]:
def review_scroller(scroll):
    scroll_attempts = 0  # 스크롤 시도 횟수
    max_attempts = 5     # 최대 시도 횟수
    number = 0

    while True:
        number = number + 1
        print(number)  # 스크롤 횟수

        # 스크롤 전 문서 높이
        last_height = driver.execute_script("return arguments[0].scrollHeight", scroll)  # 현재 문서의 높이
        print(f'last height: {last_height}')

        # 선택한 요소를 기준으로 페이지를 10000 픽셀 아래로 스크롤
        driver.execute_script('arguments[0].scrollBy(0, 10000);', scroll)
        time.sleep(2)  # 스크롤 후 대기

        # 스크롤 후 문서 높이
        new_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
        print(f'new height: {new_height}')

        if new_height == last_height:
            scroll_attempts += 1
        else:
            scroll_attempts = 0

        if scroll_attempts >= max_attempts:
            break

## 관람평 페이지

In [60]:
driver.maximize_window() 

In [36]:
driver.quit()

In [155]:
for movie in movie_list:
    naver_review_df = pd.DataFrame(columns=['keyword', 'id', 'rating', 'comment', 'date', 'up', 'down', 'type']) #type=0 if viewer, 1 if critic
    print(movie)
    get_reviews(movie)
    if '/' in str(movie): 
        movie = movie.replace('/', '-')
    naver_review_df.to_csv(f'/Users/yewonsong/desktop/reviews/2020/{movie}_naver_reviews.csv',index=True)

#살아있다
Element not found or not visible:
언힌지드
Num type review: 3
Get 실관람평
1
last height: 1657
new height: 3242
2
last height: 3242
new height: 4758
3
last height: 4758
new height: 6228
4
last height: 6228
new height: 6375
5
last height: 6375
new height: 6375
6
last height: 6375
new height: 6375
7
last height: 6375
new height: 6375
8
last height: 6375
new height: 6375
9
last height: 6375
new height: 6375
1
작성자: rkrk****
작성일: 2020.10.09.
별점: 별점(10점 만점 중)
8
감상평: 세상은 넓고 미친놈들이 많다고 하더니 진짜 이 영화를 보게 되면서 알게 되었다. 우리가 살고 있는 이런 현실에서 충분히 일어날 수 있다고 본다.
공감수: 198 비공감수: 27
리뷰 총합: 1
1
작성자: jhst****
작성일: 2020.10.10.
별점: 별점(10점 만점 중)
8
감상평: 이 영화는 앞으로 이런 일들이 일상이 될 것이라는 것을 얘기하고 싶은 것 같다. 지금과 같은 불평등, 매몰찬 경쟁, 도덕의 추락 속에서 더 이상 물러설 곳 없이 몰린 사람들이 자기 자신은 물론 자기 주변까지 파괴하고 마는 그런 일상을. 어떤 사회든 사람들이 서로 존중하고 배려하고 참지 않으면 결국 지...
공감수: 140 비공감수: 14
리뷰 총합: 2
1
작성자: yhuw****
작성일: 2020.10.11.
별점: 별점(10점 만점 중)
6
감상평: 여자주인공 겁나 답답하다;;;경찰서를 가라고!! 글래디에이터 레미제라블 러셀 크로우가 어쩌다가 이런 영화에 출연을ㅠㅠ
공감수: 145 비공감수: 25
리뷰 총합: 3
1
작성자: rty7****
작

In [153]:
def get_reviews(keyword):
    search_keyword = str(keyword) 
    base_url = "https://search.naver.com/search.naver?where=view&sm=tab_jum&query="
    search_url = base_url + search_keyword
    driver.get(search_url)
    time.sleep(3)

    try:
        specific = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[1]/section[1]/div/div/div')
        if '제외한 상세검색 결과입니다' in specific.text:
            driver.find_element(By.CSS_SELECTOR, 'a.more_link').click()
            print("일반 결과 클릭")
            time.sleep(2)
    except Exception as e:
        pass

    #관람평 페이지로 이동 
    try:
        for j in range(5): 
            element = driver.find_elements(By.CSS_SELECTOR, 'span.menu')[j]
            if '관람평' in element.text:
                element.click()
        time.sleep(2)

    except Exception as e:
        pass
    
    #리뷰 종류 개수 확인 
    try:
        menu_review = driver.find_elements(By.CSS_SELECTOR, 'ul.tab_list')[1].text.split('\n')
        num_type_review = 2 #네티즌, 평론가 
        if '실관람객' in menu_review:
            num_type_review = 3 #실관람객, 네티즌, 평론가
        print("Num type review:", num_type_review)
        #평점 부분으로 스크롤 내리기 
        ActionChains(driver).scroll_by_amount(0, 600).perform()
        time.sleep(2)
        
        #리뷰가 있는지 없는지 판단
        viewer_elements = driver.find_elements(By.CSS_SELECTOR, 'div.lego_review_list._scroller')
        there1 = viewer_elements[0].text
        TF1 = int('아직 등록된 평점이 없습니다.' in there1)

        if TF1 == 1:
            naver_review_df.loc[0] = [keyword, 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', -1]
            print("아직 등록된 평점이 없습니다")

        if TF1==0: 
            ##### 메뉴 3개인 경우 #####
            if num_type_review ==3: 
                ### [1] 실관람평 ###   
                print("Get 실관람평")
                driver.find_element(By.CSS_SELECTOR, 'button.btn_area_auto').click() #스포일러 포함
                time.sleep(1)
                scroll_element = driver.find_element(By.CSS_SELECTOR, 'div.lego_review_list._scroller')
                review_scroller(scroll_element)
                i = 1
                df_num = 1
                while True:
                    raw_comment = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[4]/div[4]/ul/li[{i}]/div[2]/div/span[2]')
                    print(len(raw_comment))
                    if len(raw_comment) == 0:
                        break
                    raw_id = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[4]/div[4]/ul/li[{i}]/dl/dd[1]')
                    raw_rating = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[4]/div[4]/ul/li[{i}]/div[1]/div/div[2]')
                    raw_date = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[4]/div[4]/ul/li[{i}]/dl/dd[2]')
                    raw_up = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[4]/div[4]/ul/li[{i}]/div[3]/button[1]/span')
                    raw_down = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[4]/div[4]/ul/li[{i}]/div[3]/button[2]/span')
                    try: 
                        id = raw_id[0].text
                    except IndexError:
                        break
                    comment = raw_comment[0].text
                    rating =  raw_rating[0].text
                    date = raw_date[0].text[0:11]
                    up = raw_up[0].text
                    down = raw_down[0].text
                    add_elements = [keyword, id, rating, comment, date, up, down, '0']
                    naver_review_df.loc[df_num] = add_elements
                    #time.sleep(0.5)
                    if len(comment) > 0:
                        print("================================================================================================")
                        print("작성자:", id)
                        print("작성일:", date)    # 작성일 출력
                        print("별점:", rating) # 별점 출력
                        print("감상평:", comment)  # 코멘트 출력
                        print("공감수:", up, end=' ')  # 공감수 출력
                        print("비공감수:", down)    # 비공감수 출력
                        print("리뷰 총합:", i)
                    i += 1
                    df_num += 1

                ### [2] 네티즌 ###  
                driver.execute_script("window.scrollTo(0, 0);")
                time.sleep(2)
                driver.find_element(By.XPATH,'//*[@id="main_pack"]/div[3]/div[2]/div/div/div[1]/div/div/ul/li[2]/a/span').click() #네티즌 페이지로 이동
                time.sleep(2)

                #평점 부분으로 스크롤 내리기 
                ActionChains(driver).scroll_by_amount(0, 600).perform()
                time.sleep(2)

                #스포일러 감상 보여주기 버튼 클릭
                driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[5]/div[2]/div[2]/button').click() #스포일러 포함
                time.sleep(2)

                scroll_element = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[3]/div[2]/div/div/div[5]/div[4]') 
                review_scroller(scroll_element)

                i = 1
                while True:
                    raw_comment = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[5]/div[4]/ul/li[{i}]/div[2]/div/span')
                    if len(raw_comment) == 0:
                        break
                    raw_id = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[5]/div[4]/ul/li[{i}]/dl/dd[1]')
                    raw_rating = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[5]/div[4]/ul/li[{i}]/div[1]/div/div[2]')
                    raw_date = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[5]/div[4]/ul/li[{i}]/dl/dd[2]')
                    raw_up = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[5]/div[4]/ul/li[{i}]/div[3]/button[1]/span')
                    raw_down = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[5]/div[4]/ul/li[{i}]/div[3]/button[2]/span')
                    try: 
                        id = raw_id[0].text
                    except IndexError:
                        break
                    comment = raw_comment[0].text
                    rating =  raw_rating[0].text
                    date = raw_date[0].text[0:11]
                    up = raw_up[0].text
                    down = raw_down[0].text
                    add_elements = [keyword, id, rating, comment, date, up, down, '1']
                    naver_review_df.loc[df_num] = add_elements
                    if len(comment) > 0:
                        print("================================================================================================")
                        print("작성자:", id)
                        print("작성일:", date)    # 작성일 출력
                        print("별점:", rating) # 별점 출력
                        print("감상평:", comment)  # 코멘트 출력
                        print("공감수:", up, end=' ')  # 공감수 출력
                        print("비공감수:", down)    # 비공감수 출력
                        print("리뷰 총합:", i)
                    df_num +=1
                    i += 1  

                ### [3] 평론가 리뷰
                driver.execute_script("window.scrollTo(0, 0);")
                time.sleep(2)
                driver.find_element(By.XPATH,f'//*[@id="main_pack"]/div[3]/div[2]/div/div/div[1]/div/div/ul/li[{num_type_review}]/a/span').click()
                time.sleep(2)

                #평점 부분으로 스크롤 내리기 
                ActionChains(driver).scroll_by_amount(0, 300).perform()
                time.sleep(2)

                scroll_element = driver.find_element(By.CSS_SELECTOR, '.lego_critic_outer._scroller')
                review_scroller(scroll_element)               
                        
                i = 1
                while True:
                    raw_comment = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[6]/div/ul/li[{i}]/div[3]/span')
                    if len(raw_comment) == 0:
                        break
                    raw_id = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[6]/div/ul/li[{i}]/div[1]/div/dl[1]/dd[1]')
                    raw_rating = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[6]/div/ul/li[{i}]/div[2]/div[2]')
                    id = raw_id[0].text
                    comment = raw_comment[0].text
                    rating =  raw_rating[0].text
                    try: 
                        raw_date = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[6]/div/ul/li[{i}]/div[1]/div/dl[2]/dd')
                        date = raw_date[0].text[0:11]
                    except IndexError:
                        raw_date = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[6]/div/ul/li[{i}]/div[1]/div/dl/dd[2]')
                        date = raw_date[0].text[0:11]
                    add_elements = [keyword, id, rating, comment, date, 'NA', 'NA', '2']
                    naver_review_df.loc[df_num] = add_elements
                    if len(comment) > 0:
                        print("================================================================================================")
                        print("작성자:", id)
                        print("작성일:", date)    # 작성일 출력
                        print("별점:", rating) # 별점 출력
                        print("감상평:", comment)  # 코멘트 출력
                        print("리뷰 총합:", i)
                    df_num += 1
                    i+=1
            
            #### 2가지일 경우 #######
            if num_type_review ==2:
                #### [1] 네티즌 리뷰 ###
                driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[3]/div[2]/div[2]/button').click() #스포일러 포함
                time.sleep(1)
                scroll_element = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[3]/div[2]/div/div/div[3]/div[4]') 
                review_scroller(scroll_element)
                print("scrolled for 2 menus")
                i = 1
                df_num = 1
                while True:
                    raw_comment = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[3]/div[4]/ul/li[{i}]/div[2]/div/span')
                    if len(raw_comment) == 0:
                        break
                    raw_id = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[3]/div[4]/ul/li[{i}]/dl/dd[1]')
                    raw_rating = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[3]/div[4]/ul/li[{i}]/div[1]/div/div[2]')
                    raw_date = driver.find_elements(By.XPATH, f'//*[@id="main_pack"]/div[3]/div[2]/div/div/div[3]/div[4]/ul/li[{i}]/dl/dd[2]')              
                    raw_up = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[3]/div[4]/ul/li[{i}]/div[3]/button[1]/span')
                    raw_down = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[3]/div[4]/ul/li[{i}]/div[3]/button[2]/span')
                    id = raw_id[0].text
                    comment = raw_comment[0].text
                    rating =  raw_rating[0].text
                    date = raw_date[0].text[0:11]
                    up = raw_up[0].text
                    down = raw_down[0].text
                    add_elements = [keyword, id, rating, comment, date, up, down, '1']
                    naver_review_df.loc[df_num] = add_elements
                    #time.sleep(0.5)
                    if len(comment) > 0:
                        print("================================================================================================")
                        print("작성자:", id)
                        print("작성일:", date)    # 작성일 출력
                        print("별점:", rating) # 별점 출력
                        print("감상평:", comment)  # 코멘트 출력
                        print("공감수:", up, end=' ')  # 공감수 출력
                        print("비공감수:", down)    # 비공감수 출력
                        print("리뷰 총합:", i)
                    i += 1
                    df_num += 1

                ### [2] 평론가 리뷰
                driver.execute_script("window.scrollTo(0, 0);")
                time.sleep(2)
                driver.find_element(By.XPATH,f'//*[@id="main_pack"]/div[3]/div[2]/div/div/div[1]/div/div/ul/li[{num_type_review}]/a/span').click()
                time.sleep(2)

                #평점 부분으로 스크롤 내리기 
                ActionChains(driver).scroll_by_amount(0, 300).perform()
                time.sleep(2)

                scroll_element = driver.find_element(By.CSS_SELECTOR, '.lego_critic_outer._scroller')
                print(len(driver.find_elements(By.CSS_SELECTOR, '.lego_critic_outer._scroller')))
                if len(driver.find_elements(By.CSS_SELECTOR, '.lego_critic_outer._scroller')) == 1:
                    raw_comment = driver.find_elements(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[4]/div/ul/li/div[3]/span')
                    raw_id = driver.find_elements(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[4]/div/ul/li/div[1]/div/dl/dd[1]')
                    raw_rating = driver.find_elements(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[4]/div/ul/li/div[2]/div[2]')
                    id = raw_id[0].text
                    comment = raw_comment[0].text
                    rating =  raw_rating[0].text
                    raw_date = driver.find_elements(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[4]/div/ul/li/div[1]/div/dl/dd[2]')
                    date = raw_date[0].text[0:11]
                    add_elements = [keyword, id, rating, comment, date, 'NA', 'NA', '2']
                    naver_review_df.loc[df_num] = add_elements
                    if len(comment) > 0:
                        print("================================================================================================")
                        print("작성자:", id)
                        print("작성일:", date)    # 작성일 출력
                        print("별점:", rating) # 별점 출력
                        print("감상평:", comment)  # 코멘트 출력
                        print("리뷰 총합:", i)        

                i = 1
                while True:
                    raw_comment = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[6]/div/ul/li[{i}]/div[3]/span')        
                    if len(raw_comment) == 0:                   
                        break
                    raw_id = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[6]/div/ul/li[{i}]/div[1]/div/dl[1]/dd[1]')
                    raw_rating = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[6]/div/ul/li[{i}]/div[2]/div[2]')
                    id = raw_id[0].text
                    comment = raw_comment[0].text
                    rating =  raw_rating[0].text
                    try: 
                        raw_date = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[6]/div/ul/li[{i}]/div[1]/div/dl[2]/dd')
                        date = raw_date[0].text[0:11]
                    except IndexError:
                        raw_date = driver.find_elements(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[1]/div[3]/div[2]/div/div/div[6]/div/ul/li[{i}]/div[1]/div/dl/dd[2]')
                        date = raw_date[0].text[0:11]
                    add_elements = [keyword, id, rating, comment, date, 'NA', 'NA', '2']
                    naver_review_df.loc[df_num] = add_elements
                    if len(comment) > 0:
                        print("================================================================================================")
                        print("작성자:", id)
                        print("작성일:", date)    # 작성일 출력
                        print("별점:", rating) # 별점 출력
                        print("감상평:", comment)  # 코멘트 출력
                        print("리뷰 총합:", i)
                    df_num += 1
                    i+=1

    except Exception as e:
    # Handle the case where the element is not found or not visible
        print("Element not found or not visible:")